このJupyter notebookを実行するにあたり、まず、
    scikit-learn
    nltk
の２つのパッケージを入手し、インストールしておいてください。

Pythonは3.6以上(fフォーマットのプリントが使える環境）を仮定しています。


そのうえで、以下の(NLTKに関する)作業を一度だけ行ってください。


python
バージョン情報などが出力されます。



例
(py310) G:>python
Python 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)] on win32
Type "help", "copyright", "credits" or "license" for more information.


import nltk
nltk.download()





ここでGUIでコーパスなどのダウンロードを促す画面が出る場合は指示に従い全部インストールしてください。その際、Download Directoryを必ずメモしておいてください。


CUIの場合、以下のようにしてコーパス等をインストールしてください。

# NLTKを使ったコーパスのロード

In [2]:
from nltk.corpus.util import LazyCorpusLoader
from nltk.corpus.reader import *

# コーパスのロード
ma_reuters = LazyCorpusLoader(
    'ma_reuters', CategorizedPlaintextCorpusReader, '(training|test).*',
    cat_file='cats.txt', encoding='ISO-8859-2')

# MA_Reutersのロード
documents = ma_reuters.fileids()
print( f"{str(len(documents))}  記事総数")

# 訓練とテストデータの文書IDの抽出
train_docs_id = [doc for doc in documents if doc.startswith("train")]
test_docs_id = [doc for doc in documents if doc.startswith("test")]

print( f"{str(len(train_docs_id))} 訓練データ")
print( f"{str(len(test_docs_id))} テストデータ")

# 訓練とテストデータの生データの抽出
train_docs = [ma_reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [ma_reuters.raw(doc_id) for doc_id in test_docs_id]

# カテゴリーのリスト
categories = ma_reuters.categories()
num_categories = len(categories)

print( f"{num_categories} カテゴリー")
print( f"{categories}")

10700  記事総数
7713 訓練データ
2987 テストデータ
55 カテゴリー
['acq', 'alum', 'barley', 'bop', 'carcass', 'cocoa', 'coffee', 'copper', 'corn', 'cotton', 'cpi', 'crude', 'dlr', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'hog', 'housing', 'interest', 'ipi', 'iron-steel', 'jobs', 'lead', 'livestock', 'meal-feed', 'money-fx', 'money-supply', 'nat-gas', 'oilseed', 'orange', 'palm-oil', 'pet-chem', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


# カテゴリーにある記事を表示


In [3]:
# 生の文書例
# Documents in a category
category_docs = ma_reuters.fileids("soybean");
document_id = category_docs[0] # オレンジ・カテゴリーの最初の文書

# 記事の中身を表示
print( f"{ma_reuters.raw(document_id)}")

JAPAN MINISTRY SAYS OPEN FARM TRADE WOULD HIT U.S.
  Japan's Agriculture Ministry, angered by
  U.S. Demands that Japan open its farm products market, will
  tell U.S. Officials at talks later this month that
  liberalisation would harm existing U.S. Farm exports to Japan,
  a senior ministry official said.
      "Imports from the U.S. Would drop due to active sales drives
  by other suppliers," the official, who declined to be named,
  said. "Japan is the largest customer for U.S. Farm products and
  it is not reasonable for the U.S. To demand Japan liberalise
  its farm import market," he said.
      Agriculture Minister Mutsuki Kato has said if the U.S.
  Insists Japan open its protected rice market it will also open
  its wheat market, where volume and origin are regulated to
  protect local farmers.
      Australia and Canada could then increase their wheat
  exports as they are more competitive than the U.S., He said.
  End-users would also buy other origins, grain traders said.


# NLTKを用いたテキストのトークン抽出, scikit-learnを用いたテキストのTF-IDFモデル抽出

In [4]:
from nltk import word_tokenize
import re # 正規表現
from sklearn.feature_extraction.text import TfidfVectorizer
 
def tokenize(text): # テキストを小文字英語トークンに変換してリストで返す関数
    min_length = 3 # 3文字以上のものだけ残す
    words = [word.lower() for word in word_tokenize(text)]
    p = re.compile('[a-zA-Z]+') # 小文字化しているが一応アルファベットで開始
    filtered_tokens = [token for token in words \
                       if p.match(token) and len(token) >= min_length]
    return filtered_tokens

# TF-IDF重みでベクトル化
vectorizer = TfidfVectorizer(stop_words='english', tokenizer=tokenize)

# 訓練データはfit_transform関数で決められた語彙に基づきTF-IDFを計算
vectorised_train_documents = vectorizer.fit_transform(train_docs)

# テストデータはtransform関数で決められた語彙に基づきTF-IDFを計算
vectorised_test_documents = vectorizer.transform(test_docs)

print(f"TF-IFモデルに変換しました")
print(f"訓練データの文書数x次元数：{vectorised_train_documents.shape}")
print(f"テストデータの文書数x次元数：{vectorised_test_documents.shape}")

TF-IFモデルに変換しました
訓練データの文書数x次元数：(7713, 26978)
テストデータの文書数x次元数：(2987, 26978)


# SVMによる分類

In [7]:
type(OVR_predictions)

numpy.ndarray

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import jaccard_score
from sklearn.metrics import hamming_loss
import numpy as np

mlb = MultiLabelBinarizer()

train_labels = mlb.fit_transform([ma_reuters.categories(doc_id)
                                  for doc_id in train_docs_id])
test_labels = mlb.transform([ma_reuters.categories(doc_id)
                             for doc_id in test_docs_id])

# マルチクラス　マルチラベル分類器で訓練＋予測
OVR_classifier = OneVsRestClassifier(LinearSVC(random_state=41))
OVR_classifier.fit(vectorised_train_documents, train_labels)
OVR_predictions = OVR_classifier.predict(vectorised_test_documents)

# Jaccard係数の計算
print(f"Jaccard係数による評価:"
      f"{np.round(jaccard_score(test_labels,OVR_predictions, average='samples'),3)}")

# Hamming損失の計算
print(f"Hamming損失による評価:"
      f"{np.round(hamming_loss(test_labels,OVR_predictions),3)}")


Jaccard係数による評価:0.86
Hamming損失による評価:0.005


In [ ]:
clen=2987
test_labels_split=[]
predictions_split=[]

for i in range(clen):
    test_labels_split.append(np.argmax(test_labels[i]))
    predictions_split.append(np.argmax(OVR_predictions[i]))

"""test_labels_split=np.empty([55,100,55])
predictions_split=np.empty([55,100,55])
for i in range(clen):
    new_index=np.argmax(test_labels[0])
    np.append(test_labels_split[new_index],test_labels[i],axis=0)
    np.append(predictions_split[new_index],OVR_predictions[new_index],axis=0)"""

'test_labels_split=np.empty([55,100,55])\npredictions_split=np.empty([55,100,55])\nfor i in range(clen):\n    new_index=np.argmax(test_labels[0])\n    np.append(test_labels_split[new_index],test_labels[i],axis=0)\n    np.append(predictions_split[new_index],OVR_predictions[new_index],axis=0)'